In [ ]:
# creates an interactive dropdown widget at the top of the notebook. Here’s a breakdown of what it does:
dbutils.widgets.dropdown("Model Stage", "None", ["Production", "Staging","None"])

In [ ]:
import mlflow

In [ ]:
registry_uri = f'databricks://modelregistery:modelregistery'
mlflow.set_registry_uri(registry_uri)

In [ ]:
model_name = "power-forecasting-model"
run_id='c9830cb0281842eb85cbb0dd9c14d896'
# The default path where the MLflow autologging function stores the model
artifact_path = "model"
model_uri = "runs:/{run_id}/{artifact_path}".format(run_id=run_id, artifact_path=artifact_path)

In [ ]:
model_production_uri = "models:/{model_name}/{model_stage}".format(model_name=model_name,model_stage=dbutils.widgets.get("Model Stage"))

In [ ]:
def plot(model_name, model_stage, model_version, power_predictions, past_power_output):
  import pandas as pd
  import matplotlib.dates as mdates
  from matplotlib import pyplot as plt
  index = power_predictions.index
  fig = plt.figure(figsize=(11, 7))
  ax = fig.add_subplot(111)
  ax.set_xlabel("Date", size=20, labelpad=20)
  ax.set_ylabel("Power\noutput\n(MW)", size=20, labelpad=60, rotation=0)
  ax.tick_params(axis='both', which='major', labelsize=17)
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
  ax.plot(index[:len(past_power_output)], past_power_output, label="True", color="red", alpha=0.5, linewidth=4)
  ax.plot(index, power_predictions.squeeze(), "--", label="Predicted by '%s'\nin stage '%s' (Version %d)" % (model_name, model_stage, model_version), color="blue", linewidth=3)
  ax.set_ylim(ymin=0, ymax=max(3500, int(max(power_predictions.values) * 1.3)))
  ax.legend(fontsize=14)
  plt.title("Wind farm power output and projections", size=24, pad=20)
  plt.tight_layout()
  display(plt.show())

In [ ]:
def forecast_power(model_name, model_stage):
  from mlflow.tracking.client import MlflowClient
  client = MlflowClient()
  model_version = client.get_latest_versions(model_name, stages=[model_stage])[0].version
  model_uri = "models:/{model_name}/{model_stage}".format(model_name=model_name, model_stage=model_stage)
  model = mlflow.pyfunc.load_model(model_uri)
  weather_data, past_power_output = get_weather_and_forecast()
  power_predictions = pd.DataFrame(model.predict(weather_data))
  power_predictions.index = pd.to_datetime(weather_data.index)
  print(power_predictions)
  plot(model_name, model_stage, int(model_version), power_predictions, past_power_output)

```python
from mlflow.tracking.client import MlflowClient client = MlflowClient()
```

This imports and creates a client to interact with MLflow’s tracking and model registry APIs.

---

```python
model = mlflow.pyfunc.load_model(model_uri)
```
Loads the model as a PyFunc model, which is MLflow’s standard format for making predictions.


### Q. which version it loads?
when you use model = mlflow.pyfunc.load_model(model_uri), you do not tell the version. which version it loads?

- you're totally right to notice that mlflow.pyfunc.load_model(model_uri) uses just the model name and stage, not an explicit version number.
- MLflow will automatically load the latest version of the model in the specified stage.

🔐 If you wanted to load a specific version (7 here):
```python
model_uri = "models:/power_forecast_model/7"
model = mlflow.pyfunc.load_model(model_uri)
```
---




In [ ]:
import pandas as pd

In [ ]:
wind_farm_data = pd.read_csv("https://github.com/dbczumar/model-registry-demo-notebook/raw/master/dataset/windfarm_data.csv", index_col=0)
def get_weather_and_forecast():
  format_date = lambda pd_date : pd_date.date().strftime("%Y-%m-%d")
  today = pd.Timestamp('today').normalize()
  week_ago = today - pd.Timedelta(days=5)
  week_later = today + pd.Timedelta(days=5)
  
  past_power_output = pd.DataFrame(wind_farm_data)[format_date(week_ago):format_date(today)]
  weather_and_forecast = pd.DataFrame(wind_farm_data)[format_date(week_ago):format_date(week_later)]
  if len(weather_and_forecast) < 10:
    past_power_output = pd.DataFrame(wind_farm_data).iloc[-10:-5]
    weather_and_forecast = pd.DataFrame(wind_farm_data).iloc[-10:]

  return weather_and_forecast.drop(columns="power"), past_power_output["power"]

In [ ]:
forecast_power(model_name, dbutils.widgets.get("Model Stage"))